In [1]:
import ee
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import agrigee_lite as agl

np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project="ee-paulagibrim")

In [3]:
df = agl.get_all_tasks()

In [ ]:
set(df.description.apply(lambda x: x.split("_", 1)[0] + "_" + x.split("_", 2)[2]).tolist())

In [4]:
gdf = gpd.read_parquet("tests/data/gdf.parquet")

In [5]:
satellite = agl.sat.Sentinel2()

In [6]:
agl.get.multiple_sits_chunks_gdrive(gdf, satellite)

{'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_5', 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_4', 'download_samples', 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_3', 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_6', 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_0', 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_2', 'agl_multiple_sits_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_1'}


100%|██████████| 256/256 [00:00<00:00, 230862.57it/s]

agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_0
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_1
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_2
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_3
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_4
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_5
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_6
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_7
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_8
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_9
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_10
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_11
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_12
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_13
agl_multiple_sits_s2_d380c64a17a6d917e4fc9a186b5b776a0bf78ac1_14
agl_multiple_sits_s2_d380c64a17a6d9

In [ ]:
from agrigee_lite.misc import create_gdf_hash

In [ ]:
hashname = create_gdf_hash(gdf)

In [ ]:
satellite = agl.sat.Sentinel2(use_sr=True, selected_bands=["red", "green", "blue"])

In [ ]:
for cluster_id in sorted(gdf.cluster_id.unique()):
    cluster_id = int(cluster_id)
    agl.get.multiple_sits_task_gdrive(gdf[gdf.cluster_id == cluster_id], satellite, f"{hashname}_{cluster_id}", f"agl_mateus_multiple_sits_{hashname}_{cluster_id}")

    if cluster_id > 5:
        break

In [ ]:
row = gdf.iloc[0]

In [ ]:
sits = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
sits

In [ ]:
sits_2

In [ ]:
df_wide = pd.DataFrame(sits_2.values.flatten().reshape(1, -1))
df_wide.columns = [f"{col}_{i}" for i in range(len(sits_2)) for col in sits_2.columns]

In [ ]:
sits = agl.get.multiple_sits(gdf.iloc[0:2], satellite)

In [ ]:
def flatten_by_indexnum(df: pd.DataFrame, prefix: str = "") -> pd.DataFrame:
    if "indexnum" not in df.columns:
        raise ValueError("A coluna 'indexnum' não existe no DataFrame.")

    df = df.copy()
    df["seq"] = df.groupby("indexnum").cumcount()
    df_wide = df.pivot(index="indexnum", columns="seq").drop(columns="indexnum")
    df_wide.columns = [f"{prefix}{col}_{i}" for col, i in df_wide.columns]
    return df_wide.reset_index()

In [ ]:
sits

In [ ]:
flatten_by_indexnum(sits)

In [ ]:
flatten_by_indexnum(sits, "a")

In [ ]:
row = gdf.iloc[0]

imgs = agl.get.images(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
np.savez_compressed("tests/data/imgs/0_s2sr.npz", data=imgs)

In [ ]:
np.load("tests/data/imgs/0_s2sr.npz")

In [ ]:
np.array_equal(imgs, imgs)

In [ ]:
imgs_copy = imgs.copy()

In [ ]:
plt.imshow((imgs[3]/10000) * np.pi)

In [ ]:
sits = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
sits.columns = [column.split("_", 1)[1] for column in sits.columns.tolist()]

In [ ]:
def remove_underscore_in_df(df):
    df.columns = [column.split("_", 1)[1] for column in df.columns.tolist()]

In [ ]:
sits.head(5)